In [1]:
# ==============================================================================
# Whisper Amharic Fine-tuning Pipeline (Final Clean Version)
# ==============================================================================

# ----------------------------
# 0) INSTALLS & ENVIRONMENT FIXES
# ----------------------------
print("Step 0: Installing required libraries...")
!pip install -q "protobuf==3.20.3"
!pip install -q transformers datasets evaluate soundfile librosa accelerate jiwer
!pip install torchcodec
!pip install tqdm
print("Installation complete.")

Step 0: Installing required libraries...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
pyd

In [2]:

# ----------------------------
# 1) IMPORTS
# ----------------------------
import os
import re
import numpy as np
import pandas as pd
import soundfile as sf
import librosa
import torch
from dataclasses import dataclass
from typing import Any, Dict, List
from datasets import load_dataset, Dataset, DatasetDict, Audio
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
from transformers import logging as hf_logging
from evaluate import load as load_metric

hf_logging.set_verbosity_error()


2025-12-05 13:54:40.208130: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764942880.393223      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764942880.444872      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# ----------------------------
# 2) CONFIGURATION
# ----------------------------
print("\nStep 1: Setting up configuration...")

# --- DATASET PATH ---
# These paths are confirmed to be correct.
DATASET_BASE_PATH = "/kaggle/input/amharic-speech-corpus"
DATASET_ROOT_DIR = os.path.join(DATASET_BASE_PATH, "AMHARIC")
TRAIN_DATA_DIR = os.path.join(DATASET_ROOT_DIR, "data", "train")
TEST_DATA_DIR = os.path.join(DATASET_ROOT_DIR, "data", "test")

# --- MODEL & OUTPUT SETTINGS ---
MODEL_NAME = "openai/whisper-base"
OUTPUT_DIR = "./whisper-amharic-finetuned"
PROCESSED_DS_DIR = "./processed_whisper_amharic"

# --- TRAINING PARAMETERS ---
SAMPLE_RATE = 16000
MAX_AUDIO_SECONDS = 30.0
BATCH_SIZE = 8
NUM_EPOCHS = 6
LEARNING_RATE = 1e-5
ACCUMULATION_STEPS = 2
SEED = 42
NUM_PROC = 4

# --- LOGGING & SAVING ---
USE_FP16 = True
LOGGING_STEPS = 50
EVAL_STEPS = 500
SAVE_STEPS = 500

# Create output directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(PROCESSED_DS_DIR, exist_ok=True)

print("Configuration set. Paths verified.")
print(f"  - Train Data: {TRAIN_DATA_DIR}")
print(f"  - Test Data:  {TEST_DATA_DIR}")



Step 1: Setting up configuration...
Configuration set. Paths verified.
  - Train Data: /kaggle/input/amharic-speech-corpus/AMHARIC/data/train
  - Test Data:  /kaggle/input/amharic-speech-corpus/AMHARIC/data/test


In [4]:
# ----------------------------
# 3) UTILITY FUNCTIONS
# ----------------------------
def normalize_amharic_text(text):
    if text is None: return ""
    s = str(text)
    s = re.sub(r"[A-Za-z]", "", s)
    s = re.sub(r"[\u0000-\u007F]+", " ", s)
    s = re.sub(r"[!\"#\$%&\(\)\*\+,\-\.\/:;<=>\?@\[\]\\\^_`{\|}~]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def load_kaldi_data(data_dir):
    """Loads data from a Kaldi-style directory and returns a Pandas DataFrame."""
    print(f"\n--- Loading data from: {data_dir} ---")
    wav_scp_path = os.path.join(data_dir, "wav.scp")
    text_path = os.path.join(data_dir, "text")

    if not os.path.exists(wav_scp_path):
        print(f"ERROR: wav.scp not found at {wav_scp_path}")
        return pd.DataFrame()
    if not os.path.exists(text_path):
        print(f"ERROR: text not found at {text_path}")
        return pd.DataFrame()

    wav_scp = {}
    with open(wav_scp_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(' ', 1)
            if len(parts) == 2:
                file_id, wav_path = parts
                if not os.path.isabs(wav_path):
                    wav_path = os.path.join(data_dir, wav_path)
                wav_scp[file_id] = wav_path
    
    text = {}
    with open(text_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(' ', 1)
            if len(parts) == 2:
                file_id, transcription = parts
                text[file_id] = transcription
    
    print(f"Found {len(wav_scp)} entries in wav.scp")
    print(f"Found {len(text)} entries in text")

    data = []
    for file_id, path in wav_scp.items():
        if file_id in text:
            data.append({'path': path, 'sentence': text[file_id]})
            
    print(f"Successfully matched {len(data)} entries.")
    print("--- Data loading finished ---")
            
    return pd.DataFrame(data)

In [5]:
# ==============================================================================
# CELL 4: DATA LOADING AND PREPARATION (Modified to fix error)
# ==============================================================================

# --- Imports needed for this cell ---
import os
import re
import pandas as pd
from datasets import Dataset, DatasetDict # NO LONGER NEED Audio import here

# --- Configuration variables needed for this cell ---
DATASET_BASE_PATH = "/kaggle/input/amharic-speech-corpus"
DATASET_ROOT_DIR = os.path.join(DATASET_BASE_PATH, "AMHARIC")
TRAIN_DATA_DIR = os.path.join(DATASET_ROOT_DIR, "data", "train")
TEST_DATA_DIR = os.path.join(DATASET_ROOT_DIR, "data", "test")
# SAMPLE_RATE = 16000 # Not needed here anymore

# --- Utility functions needed for this cell ---
def normalize_amharic_text(text):
    if text is None: return ""
    s = str(text)
    s = re.sub(r"[A-Za-z]", "", s)
    s = re.sub(r"[\u0000-\u007F]+", " ", s)
    s = re.sub(r"[!\"#\$%&\(\)\*\+,\-\.\/:;<=>\?@\[\]\\\^_`{\|}~]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def load_kaldi_data(data_dir):
    """Loads data from a Kaldi-style directory and returns a Pandas DataFrame."""
    print(f"--- Loading data from: {data_dir} ---")
    wav_scp_path = os.path.join(data_dir, "wav.scp")
    text_path = os.path.join(data_dir, "text")
    
    wav_scp = {}
    with open(wav_scp_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t', 1)
            if len(parts) == 2:
                file_id = parts[0]
                filename = f"{file_id}.wav"
                wav_path = os.path.join(data_dir, "wav", filename)
                wav_scp[file_id] = wav_path
    
    text = {}
    with open(text_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(' ', 1)
            if len(parts) == 2:
                file_id, transcription = parts
                text[file_id] = transcription
    
    print(f"Found {len(wav_scp)} entries in wav.scp and {len(text)} in text.")
    data = []
    for file_id, path in wav_scp.items():
        if file_id in text:
            data.append({'path': path, 'sentence': text[file_id]})
    print(f"Successfully matched {len(data)} entries.")
    return pd.DataFrame(data)

# --- Main data loading logic for this cell ---
print("Step 4: Loading and preparing the dataset...")
train_df = load_kaldi_data(TRAIN_DATA_DIR)
test_df = load_kaldi_data(TEST_DATA_DIR)

for df in [train_df, test_df]:
    df['sentence'] = df['sentence'].apply(normalize_amharic_text)
    df = df[df['path'].apply(os.path.exists)]
    df = df[df['sentence'].str.strip().astype(bool)]
    df.reset_index(drop=True, inplace=True)

val_df = test_df
print(f"Final dataset sizes: Train={len(train_df)}, Validation={len(val_df)}")

def make_hf_dataset(df):
    # --- CHANGE: Create a simple 'audio' column with just the path ---
    recs = [{"audio": p, "sentence": s} for p, s in zip(df['path'], df['sentence'])]
    return Dataset.from_list(recs)

train_ds = make_hf_dataset(train_df)
val_ds = make_hf_dataset(val_df)

# --- IMPORTANT CHANGE: REMOVE THE .cast_column() CALL ---
# We will handle audio loading manually in the next step.
# train_ds = train_ds.cast_column("audio", Audio(sampling_rate=SAMPLE_RATE))
# val_ds = val_ds.cast_column("audio", Audio(sampling_rate=SAMPLE_RATE))

dataset = DatasetDict({"train": train_ds, "validation": val_ds})
print("\n✅ Cell 4 finished successfully! The 'dataset' variable is ready.")

Step 4: Loading and preparing the dataset...
--- Loading data from: /kaggle/input/amharic-speech-corpus/AMHARIC/data/train ---
Found 10875 entries in wav.scp and 10875 in text.
Successfully matched 10875 entries.
--- Loading data from: /kaggle/input/amharic-speech-corpus/AMHARIC/data/test ---
Found 359 entries in wav.scp and 359 in text.
Successfully matched 359 entries.
Final dataset sizes: Train=10875, Validation=359

✅ Cell 4 finished successfully! The 'dataset' variable is ready.


In [6]:
# Let's inspect the first 10 lines of the wav.scp files
TRAIN_WAV_SCP = "/kaggle/input/amharic-speech-corpus/AMHARIC/data/train/wav.scp"
TEST_WAV_SCP = "/kaggle/input/amharic-speech-corpus/AMHARIC/data/test/wav.scp"

print("First 10 lines of TRAIN wav.scp:")
!head -10 {TRAIN_WAV_SCP}

print("\n" + "="*40 + "\n")

print("First 10 lines of TEST wav.scp:")
!head -10 {TEST_WAV_SCP}


First 10 lines of TRAIN wav.scp:
tr_10000_tr097082	/home/melese/kaldi/data/train/wav/tr_10000_tr097082.wav
tr_10001_tr097083	/home/melese/kaldi/data/train/wav/tr_10001_tr097083.wav
tr_10002_tr097084	/home/melese/kaldi/data/train/wav/tr_10002_tr097084.wav
tr_10003_tr097085	/home/melese/kaldi/data/train/wav/tr_10003_tr097085.wav
tr_10004_tr097086	/home/melese/kaldi/data/train/wav/tr_10004_tr097086.wav
tr_10005_tr097087	/home/melese/kaldi/data/train/wav/tr_10005_tr097087.wav
tr_10006_tr097088	/home/melese/kaldi/data/train/wav/tr_10006_tr097088.wav
tr_10007_tr097089	/home/melese/kaldi/data/train/wav/tr_10007_tr097089.wav
tr_10008_tr097090	/home/melese/kaldi/data/train/wav/tr_10008_tr097090.wav
tr_10009_tr097091	/home/melese/kaldi/data/train/wav/tr_10009_tr097091.wav


First 10 lines of TEST wav.scp:
01_d501021	/home/melese/kaldi/data/test/wav/01_d501021.wav
01_d501022	/home/melese/kaldi/data/test/wav/01_d501022.wav
01_d501023	/home/melese/kaldi/data/test/wav/01_d501023.wav
01_d501024	/home

In [7]:
# ==============================================================================
# CELL 5: PROCESSOR AND MODEL SETUP (Detailed Version)
# ==============================================================================

# --- Imports needed for this cell ---
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# --- Configuration variables needed for this cell ---
# (These should be defined in your earlier configuration cell)
MODEL_NAME = "openai/whisper-base"

# --- Main model loading logic for this cell ---
print("Step 5: Loading processor and model...")

# 1. Load the processor (handles tokenization and feature extraction)
print("  -> Loading WhisperProcessor...")
processor = WhisperProcessor.from_pretrained(MODEL_NAME, language="amharic", task="transcribe")
print("  -> Processor loaded successfully.")

# 2. Load the pre-trained model
print("  -> Loading WhisperForConditionalGeneration model...")
print("     (This may take a few minutes the first time as it downloads the model weights)")
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
print("  -> Model loaded successfully.")

# 3. Configure the model for Amharic transcription
print("  -> Configuring model for Amharic...")
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="amharic", task="transcribe")
model.gradient_checkpointing_enable() # Saves memory
print("  -> Model configured.")

# 4. Move the model to the GPU if available
if torch.cuda.is_available():
    print("  -> Moving model to GPU...")
    model = model.to("cuda")
    print("  -> Model moved to GPU.")
else:
    print("  -> CUDA not available, using CPU.")

print("\n✅ Cell 5 finished successfully! Processor and model are ready.")


Step 5: Loading processor and model...
  -> Loading WhisperProcessor...


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

  -> Processor loaded successfully.
  -> Loading WhisperForConditionalGeneration model...
     (This may take a few minutes the first time as it downloads the model weights)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

  -> Model loaded successfully.
  -> Configuring model for Amharic...
  -> Model configured.
  -> Moving model to GPU...
  -> Model moved to GPU.

✅ Cell 5 finished successfully! Processor and model are ready.


In [8]:
# ==============================================================================
# CELL 6: DATA PREPROCESSING (Generator Version - Final Fix)
# ==============================================================================

# --- Imports needed for this cell ---
import os
import numpy as np
import soundfile as sf
import librosa
from datasets import Dataset, DatasetDict, load_from_disk

# --- Configuration and variables needed for this cell ---
PROCESSED_DS_DIR = "./processed_whisper_amharic"
MAX_AUDIO_SECONDS = 30.0
SAMPLE_RATE = 16000

# These variables should be available from running cells 4 and 5
dataset = dataset
processor = processor

# --- The preprocessing function (no changes needed here) ---
def prepare_example(example):
    audio_path = example["audio"]
    speech_array, sr = sf.read(audio_path)
    if sr != SAMPLE_RATE:
        speech_array = librosa.resample(np.asarray(speech_array, dtype=np.float32), orig_sr=sr, target_sr=SAMPLE_RATE)
    if speech_array.ndim > 1:
        speech_array = np.mean(speech_array, axis=1)
    max_len = int(MAX_AUDIO_SECONDS * SAMPLE_RATE)
    if len(speech_array) > max_len:
        speech_array = speech_array[:max_len]
    input_features = processor.feature_extractor(speech_array, sampling_rate=SAMPLE_RATE).input_features[0]
    labels = processor(text=example["sentence"], text_target=example["sentence"]).input_ids
    return {"input_features": input_features, "labels": labels}

# --- NEW: A generator function to yield examples one by one ---
def gen_processed_examples(ds):
    """
    This generator yields processed examples one at a time,
    which is very memory-efficient.
    """
    for example in ds:
        yield prepare_example(example)

# --- Main preprocessing logic for this cell ---
print("\nStep 6: Preprocessing audio and text data using a generator...")

if not os.path.exists(os.path.join(PROCESSED_DS_DIR, "dataset_dict.json")):
    print("Processing dataset for the first time. This will take a while...")
    
    # Create datasets from the generators. This avoids memory overflow.
    print("Creating training dataset from generator...")
    train_ds = Dataset.from_generator(gen_processed_examples, gen_kwargs={"ds": dataset["train"]})
    
    print("Creating validation dataset from generator...")
    val_ds = Dataset.from_generator(gen_processed_examples, gen_kwargs={"ds": dataset["validation"]})
    
    # Combine into a DatasetDict
    proc_ds = DatasetDict({"train": train_ds, "validation": val_ds})
    
    # Save the processed dataset to disk
    print("Saving processed dataset to disk...")
    proc_ds.save_to_disk(PROCESSED_DS_DIR)
    print("Processing complete and saved to disk.")
    
else:
    print("Found processed dataset on disk. Loading from cache...")
    proc_ds = load_from_disk(PROCESSED_DS_DIR)
    print("Loading from disk complete.")

print("\n✅ Cell 6 finished successfully! Processed data is ready for training.")


Step 6: Preprocessing audio and text data using a generator...
Processing dataset for the first time. This will take a while...
Creating training dataset from generator...


Generating train split: 0 examples [00:00, ? examples/s]

Creating validation dataset from generator...


Generating train split: 0 examples [00:00, ? examples/s]

Saving processed dataset to disk...


Saving the dataset (0/21 shards):   0%|          | 0/10875 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/359 [00:00<?, ? examples/s]

Processing complete and saved to disk.

✅ Cell 6 finished successfully! Processed data is ready for training.


In [9]:
# ==============================================================================
# CELL 7: DATA COLLATOR AND METRICS
# ==============================================================================

# --- Imports needed for this cell ---
from dataclasses import dataclass
from typing import Any, Dict, List
from evaluate import load as load_metric

# --- Variables needed for this cell ---
# These should be available from running cells 4, 5, and 6
processor = processor

# ==============================================================================
# PART 1: DATA COLLATOR
# This function takes a batch of examples and pads them so they are all the same size.
# ==============================================================================
@dataclass
class DataCollatorSpeechSeq2Seq:
    processor: WhisperProcessor

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        # Pad the input features (audio data)
        input_features = [{"input_features": f["input_features"]} for f in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Pad the labels (text data) and replace padding token id with -100
        labels = [f["labels"] for f in features]
        labels_batch = self.processor.tokenizer.pad({"input_ids": labels}, return_tensors="pt")
        
        # The -100 tells the model to ignore these tokens when calculating the loss
        labels_batch["input_ids"] = np.where(
            labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id, -100, labels_batch["input_ids"]
        )
        
        batch["labels"] = torch.tensor(labels_batch["input_ids"])
        return batch

# Create an instance of our data collator
data_collator = DataCollatorSpeechSeq2Seq(processor=processor)

# ==============================================================================
# PART 2: METRICS
# This function calculates the Word Error Rate (WER) to see how well the model is doing.
# ==============================================================================
wer_metric = load_metric("wer")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    # Replace -100 with the pad token id so we can decode the labels
    labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
    
    # Decode the predicted and label IDs back into text
    decoded_preds = processor.tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Calculate the Word Error Rate
    wer = wer_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"wer": wer}

print("\n✅ Cell 7 finished successfully! Data collator and metrics are ready.")


✅ Cell 7 finished successfully! Data collator and metrics are ready.


In [ ]:
# ==============================================================================
# CELL 8: TRAINING THE MODEL (Corrected)
# ==============================================================================

# --- Imports needed for this cell ---
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# --- Variables needed for this cell ---
# These should be available from running cells 4, 5, 6, and 7
proc_ds = proc_ds
model = model
data_collator = data_collator
compute_metrics = compute_metrics

# ==============================================================================
# PART 1: SET UP TRAINING ARGUMENTS
# These arguments control the entire training process.
# ==============================================================================
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-amharic-finetuned",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=50,
    learning_rate=1e-5,
    num_train_epochs=6,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=True, # Use mixed-precision if a GPU is available
    report_to=None, # Set to "wandb" if you want to use Weights & Biases
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    save_total_limit=3,
)

# ==============================================================================
# PART 2: INITIALIZE THE TRAINER
# The trainer orchestrates the entire training and evaluation loop.
# ==============================================================================
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=proc_ds["train"],
    eval_dataset=proc_ds["validation"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics,
)

# ==============================================================================
# PART 3: START TRAINING!
# This is the final command that begins the fine-tuning process.
# ==============================================================================
print("\n🚀 Starting the training process... 🚀")
trainer.train()

# ==============================================================================
# PART 4: SAVE THE FINAL MODEL
# After training, save the best model and the processor.
# ==============================================================================
print("\n✅ Training finished! Saving the final model...")
trainer.save_model("./whisper-amharic-finetuned")
processor.save_pretrained("./whisper-amharic-finetuned")
print("Model and processor saved to './whisper-amharic-finetuned'")

/tmp/ipykernel_47/4118250548.py:44: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was


🚀 Starting the training process... 🚀


<IPython.core.display.Javascript object>